In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import seaborn as sns
import plotly.express as px

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/netflix-shows/netflix_titles.csv


In [2]:
df=pd.read_csv("../input/netflix-shows/netflix_titles.csv")

In [3]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [4]:
df=df.drop('show_id',axis=1)

### Countries With Maximum Content On Netflix

In [5]:
country_sh_mv=df['country'].value_counts()[:10]
fig=px.bar(x=country_sh_mv.index,y=country_sh_mv.values,color=country_sh_mv.index,title="Counties Having Maximum Content on Netflix")
fig.show()

### Total Contents And Their Ratings

In [6]:
z = df.groupby(['rating']).size().reset_index(name='counts')
fig = px.scatter(z, x="counts",size="counts",
             color="rating",
                 log_x=True, size_max=60,title="Total Content Rating")
fig.show()

In [7]:
fig = px.pie(z, values='counts', names='rating', title='Netflix Content Ratings')
fig.show()

### Top 5 Directors On Netflix

In [8]:
df['director']=df['director'].fillna('No Director Specified')
filtered_directors=pd.DataFrame()

In [9]:
filtered_directors=df['director'].str.split(',',expand=True).stack()
filtered_directors=filtered_directors.to_frame()
filtered_directors.columns=['Director']
directors=filtered_directors.groupby(['Director']).size().reset_index(name='Total Content')
directors=directors[directors.Director !='No Director Specified']
directors=directors.sort_values(by=['Total Content'],ascending=False)
directorsTop5=directors.head()
directorsTop5=directorsTop5.sort_values(by=['Total Content'])
fig1=px.bar(directorsTop5,x='Total Content',y='Director',title='Top 5 Directors on Netflix',template="plotly_dark")
fig1.show()

### Top 5 Actors on Netflix

In [10]:
df['cast']=df['cast'].fillna('No Cast Specified')
filtered_cast=pd.DataFrame()
filtered_cast=df['cast'].str.split(',',expand=True).stack()
filtered_cast=filtered_cast.to_frame()
filtered_cast.columns=['Actor']
actors=filtered_cast.groupby(['Actor']).size().reset_index(name='Total Content')
actors=actors[actors.Actor !='No Cast Specified']
actors=actors.sort_values(by=['Total Content'],ascending=False)
actorsTop5=actors.head()
actorsTop5=actorsTop5.sort_values(by=['Total Content'])


In [11]:
fig2=px.bar(actorsTop5,x='Total Content',y='Actor',title='Top 5 Actors on Netflix',color='Actor',
            template="ggplot2")
fig2.show()

### Trend of content produced over the years on netflix

In [12]:
df1=df[['type','release_year']]
df1=df1.rename(columns={"release_year": "Release Year"})
df2=df1.groupby(['Release Year','type']).size().reset_index(name='Total Content')
df2=df2[df2['Release Year']>=2010]
fig3 = px.line(df2, x="Release Year", y="Total Content", color='type',title='Trend of content produced over the years on netflix'
              ,template= "plotly_dark")
fig3.show()

### Longest Movies On Netflix

In [13]:
longest=df[df['type']=='Movie'][['title','duration']]
dut=longest['duration'].str.split(" ",expand=True)
longest=longest.join(dut)
longest=longest.drop('duration',axis=1)
longest=longest.drop(1,axis=1)
longest=longest.rename(columns = {0:'Duration','title':'Title'})
##longest.reset_index(drop=True,inplace=True)
longest=longest.dropna()
longest['Duration'] = longest['Duration'].astype(str).astype(int)
longest=longest.sort_values(by='Duration',ascending=False)
##longest.reset_index(drop=True,inplace=True)
fig3=px.bar(longest.head(10),x='Title',y='Duration',color='Title',title='Longest Movies On Netflix')
fig3.show()